# A&E attendence overview

Emis-Web has a population reporting sections were reports can be run based on certain criteria. A datadump of recent "Accident and emergancy" is pulled from the population reporting. The data is assumed to always be in this format:



![title](sample.png)

The goal is to visualise this data and see how frequent A&E attenders are and see the spread of the age group that are attending.

In [1]:
import pandas as pd
import sys
import datetime as dt
from datetime import datetime 
from datetime import datetime, time, date, timedelta



#Open the datafile and load into a dataframe:
df = pd.read_csv("data.csv")
df.info()
print("\n\nFirst section of the data: ")
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
Age                      12 non-null float64
Anonymised Identifier    12 non-null object
Read Code                12 non-null object
Date                     12 non-null object
Code Term                12 non-null object
Unnamed: 5               0 non-null float64
dtypes: float64(2), object(4)
memory usage: 704.0+ bytes


First section of the data: 


,Age,Anonymised Identifier,Read Code,Date,Code Term,Unnamed: 5
0,1.0,9abaf183-2932-0a30-fe4d-78c804c5ca4a,9N19-1,11-Jun-2019,Seen in accident and emergency department,NaN
1,2.0,f9365f17-50fb-02c7-b087-4dde5e040631,9N19-1,11-Jun-2019,Seen in accident and emergency department,NaN
2,4.0,39372521-0ca0-03cb-bc1e-5198941cc1dc,9N19-1,09-Jun-2019,Seen in accident and emergency department,NaN
3,6.0,dedf0f25-3c3d-12ac-fd7a-48d18c001757,9N19-1,10-Jun-2019,Seen in accident and emergency department,NaN
4,6.0,dedf0f25-3c3d-12ac-fd7a-48d18c001757,9N19-1,10-Jun-2019,Seen in accident and emergency department,NaN


At a first glance, things we need to clean:
- remove this "Unamed:5" column
- remove any NaN rows
- "Age" column. Turn this into integer
- rename "Anonymised Identifier" to "ID", something easier to manage. 
- Check if "Date" column is in format: datetime

# Cleaning the data

In [2]:
#Drop the "Unnamed:5" column 
del df["Unnamed: 5"]

#Drop any NaN data. Shouldnt be any. 
df = df.dropna()

#fixing the age column:
df = df.astype({"Age": int})

#rename "Anonymised Identifier" to "ID"
df = df.rename(columns={'Anonymised Identifier': 'ID'})

#convert "date" column into datetime from string
df['Date'] =  pd.to_datetime(df['Date'], format='%d-%b-%Y')

#df.info()
#print("\n\nFirst section of the data: ")
#df.head()


# Prepare data for visualization
First we will find the min_date and max_date. 
Then perhaps create a function to allow the user to choose their own range, else default to given data range
create new df to store filtered data containing:
| incremental dates  |  count for that date  | [list of ages]

Next we will have a list of incremental dates and loop through them. for each date:
- filter by that date and:
    - grab count 
    - grab list of ages 
Then we can use this new df to visualise the data. 